<a target="_blank" href="https://colab.research.google.com/github/jackma-00/house-price-prediction/blob/main/experiments/1_house_price_feature_backfill.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# <span style="font-width:bold; font-size: 3rem; color:#333;">Feature Backfill<span>

Installing required packages

In [1]:
!pip install hopsworks[python]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.3/640.3 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Imports

In [2]:
import pandas as pd
import hopsworks
import os

import warnings
warnings.filterwarnings("ignore")

### <span style='color:#ff5f27'> Connect to Hopsworks Feature Store

In [3]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
os.environ["HOPSWORKS_API_KEY"] = ""
proj = hopsworks.login()


Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1158295


## <span style="color:#ff5f27;"> 💽 Loading the Data </span>

In [4]:
# Connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# unzip properties.zip
!unzip /content/drive/MyDrive/properties.zip -d /content/

Archive:  /content/drive/MyDrive/properties.zip
  inflating: /content/properties.csv  


In [ ]:
# unzip omi.zip
!unzip /content/drive/MyDrive/omi.zip -d /content/

Archive:  /content/drive/MyDrive/omi.zip
  inflating: /content/omi.csv        


In [6]:
# Load properties CSV into Dataframe
properties_csv = 'properties.csv'
properties_df = pd.read_csv(properties_csv)

In [ ]:
# Load omi CSV into Dataframe
omi_csv = 'omi.csv'
omi_df = pd.read_csv(omi_csv)

In [7]:
properties_df

,id,typologyId,categoryCode,codcom,agencyId,latitude,longitude,id_zona_omi,city,zona,...,GA4Heating,hasElevator,isNew,isLuxury,description,photos,scraping_date,createdAt,updatedAt,categoryName
0,109230993,14,1,1272.0,NaN,45.082779,7.644110,L219-D8,Torino,"Pozzo Strada, Parella",...,Centralizzato,NaN,True,False,Bilocale pronto da abitare situato a 5 minuti ...,"[{'id': 1471294881, 'caption': '', 'urls': {'s...",2024-03-18,1.707222e+09,1.709585e+09,NaN
1,108895067,14,1,1272.0,178139.0,45.083300,7.645500,L219-D8,Torino,"Pozzo Strada, Parella",...,Centralizzato,NaN,False,False,ZONA CAMPIDOGLIO - VIA MICHELE LESSONA - In pi...,"[{'id': 1465033121, 'caption': 'facciata', 'ur...",2024-03-18,1.706099e+09,1.709984e+09,NaN
2,109784457,14,1,1272.0,NaN,45.058979,7.683780,L219-B8,Torino,San Salvario,...,Centralizzato,NaN,True,False,"Piazza madama, a 5 minuti dal Parco del Valent...","[{'id': 1481631325, 'caption': '', 'urls': {'s...",2024-03-18,1.709151e+09,1.710337e+09,NaN
3,108547953,14,1,1272.0,392458.0,45.057200,7.678300,L219-B8,Torino,San Salvario,...,Autonomo,NaN,False,False,MONOLOCALE IN AFFITTO ARREDATO\nRISCALDAMENTO ...,"[{'id': 1458345959, 'caption': '', 'urls': {'s...",2024-03-18,1.704902e+09,1.710408e+09,NaN
4,109914147,14,1,23059.0,229787.0,45.449700,10.668200,G489-B2,Peschiera del Garda,Centro,...,Autonomo,NaN,True,False,Per informazioni ELENA 393/8205511\nIn zona Sa...,"[{'id': 1484040585, 'caption': '', 'urls': {'s...",2024-03-25,1.709641e+09,1.709642e+09,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211904,110000113,17,1,1272.0,74278.0,45.090100,7.698500,L219-C10,Torino,"Aurora, Barriera di Milano, Rebaudengo",...,Autonomo,NaN,True,False,In via Saverio Mercadante affittiamo partico...,"[{'id': 1485646521, 'caption': 'cortile', 'url...",2024-03-18,1.709894e+09,1.709895e+09,NaN
1211905,109942307,14,1,1272.0,NaN,45.092331,7.699990,L219-D12,Torino,"Aurora, Barriera di Milano, Rebaudengo",...,Centralizzato,NaN,True,False,"Alloggio in buone condizioni, abitabile, espos...","[{'id': 1484556023, 'caption': '', 'urls': {'s...",2024-03-18,1.709718e+09,1.710158e+09,NaN
1211906,108302933,14,1,1272.0,NaN,45.091194,7.696717,L219-C10,Torino,"Aurora, Barriera di Milano, Rebaudengo",...,Autonomo,NaN,False,False,"Privato affitta trilocale di 65 mq , zona Barr...","[{'id': 1452992687, 'caption': '', 'urls': {'s...",2024-03-18,1.703504e+09,1.704724e+09,NaN
1211907,110019089,14,1,48017.0,247117.0,43.766900,11.251600,D612-B5,Firenze,Oltrarno,...,Autonomo,NaN,True,False,This wonderful 1BR apartment is in a fantastic...,"[{'id': 1486030391, 'caption': 'Interno 1', 'u...",2024-03-15,1.709841e+09,1.710492e+09,NaN


In [ ]:
omi_df

## <span style="color:#ff5f27;"> Data Cleaning

In [8]:
properties_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1211909 entries, 0 to 1211908
Data columns (total 44 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   id                   1211909 non-null  int64  
 1   typologyId           1211909 non-null  int64  
 2   categoryCode         1211909 non-null  int64  
 3   codcom               1209642 non-null  float64
 4   agencyId             1110765 non-null  float64
 5   latitude             1211909 non-null  float64
 6   longitude            1211909 non-null  float64
 7   id_zona_omi          1206438 non-null  object 
 8   city                 1211710 non-null  object 
 9   zona                 915626 non-null   object 
 10  quartiere            438259 non-null   object 
 11  name                 1211909 non-null  object 
 12  contract             1211909 non-null  object 
 13  surface              1211909 non-null  int64  
 14  condition            1120461 non-null  object 
 15

In [ ]:
omi_df.info()

Drop null values

In [9]:
properties_df.dropna(inplace=True)

In [ ]:
omi_df.dropna(inplace=True)

Convert column names to lowercase

In [10]:
# Convert column names to lowercase
properties_df.columns = properties_df.columns.str.lower()

In [ ]:
# Convert column names to lowercase
omi_df.columns = omi_df.columns.str.lower()

**Evaluating candidate features for properties fg**

Primary key:
* id

Timestamp:
* scraping_date

Numerical features:
* agencyid
* bedroomsnumber
* buildingyear
* codcom
* gsm
* surface
* latitude
* longitude

Boolean features:
* isluxury
* isnew
* on_the_market
* zeroenergybuilding

Categorical features:
* airconditioning
* bathrooms
* city
* condition
* energyclass
* ga4heating
* garage
* heatingtype
* pricerange
* rooms
* id_zona_omi

Label:
* price

In [11]:
# properties selected columns
properties_df = properties_df[[
    'id',
    'scraping_date',
    'agencyid',
    'bedroomsnumber',
    'buildingyear',
    'codcom',
    'gsm',
    'surface',
    'latitude',
    'longitude',
    'isluxury',
    'isnew',
    'on_the_market',
    'zeroenergybuilding',
    'airconditioning',
    'bathrooms',
    'city',
    'condition',
    'energyclass',
    'ga4heating',
    'garage',
    'heatingtype',
    'pricerange',
    'rooms',
    'id_zona_omi',
    'price',
]]

In [12]:
# Convert boolean features to int
properties_df['isluxury'] = properties_df['isluxury'].astype(int)
properties_df['isnew'] = properties_df['isnew'].astype(int)
properties_df['on_the_market'] = properties_df['on_the_market'].astype(int)
properties_df['zeroenergybuilding'] = properties_df['zeroenergybuilding'].astype(int)

In [13]:
properties_df['scraping_date'] = pd.to_datetime(properties_df['scraping_date'])
properties_df = properties_df.sort_values(by=['scraping_date'])

In [14]:
properties_df

,id,scraping_date,agencyid,bedroomsnumber,buildingyear,codcom,gsm,surface,latitude,longitude,...,city,condition,energyclass,ga4heating,garage,heatingtype,pricerange,rooms,id_zona_omi,price
669203,103007050,2023-09-26,139450.0,3.0,1970.0,99013.0,140.0,110,44.0007,12.6411,...,Riccione,Buono / Abitabile,G,Autonomo,1 in box privato/box in garage,"autonomo, a radiatori, alimentato a metano",200.001 - 300.000 &euro;,4,H274-D2,275000.0
656118,105802031,2023-09-26,235162.0,1.0,1975.0,37006.0,60.0,42,44.5228,11.2949,...,Bologna,Ottimo / Ristrutturato,D,Autonomo,1 in parcheggio/garage comune,"autonomo, a radiatori, alimentato a metano",100.001 - 150.000 &euro;,2,A944-E5,143500.0
656122,104049465,2023-09-26,184442.0,1.0,2005.0,37006.0,97.0,56,44.5224,11.2723,...,Bologna,Ottimo / Ristrutturato,E,Centralizzato,1 in parcheggio/garage comune,"centralizzato, a radiatori, alimentato a gas",150.001 - 200.000 &euro;,2,A944-E5,168000.0
656175,105766201,2023-09-26,17358.0,1.0,2002.0,46017.0,17.0,48,43.8420,10.4887,...,Lucca,Ottimo / Ristrutturato,E,Autonomo,1 in parcheggio/garage comune,"autonomo, a radiatori",150.001 - 200.000 &euro;,3,E715-C1,155000.0
656179,104641325,2023-09-26,375774.0,3.0,1970.0,46017.0,76.0,110,43.8423,10.4793,...,Lucca,Buono / Abitabile,G,Centralizzato,1 in parcheggio/garage comune,"centralizzato, a radiatori, alimentato a metano",100.001 - 150.000 &euro;,5+,E715-D1,149000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530585,99346912,2024-04-01,223338.0,2.0,1970.0,36023.0,508.0,160,44.6267,10.9078,...,Modena,Ottimo / Ristrutturato,F,Autonomo,"1 in box privato/box in garage, 1 in parcheggi...","autonomo, a radiatori, alimentato a gas",200.001 - 300.000 &euro;,5,F257-D28,298000.0
530586,103521532,2024-04-01,374571.0,2.0,1967.0,36023.0,419.0,70,44.6375,10.9087,...,Modena,Buono / Abitabile,G,Autonomo,1 in box privato/box in garage,"autonomo, a radiatori, alimentato a gas",150.001 - 200.000 &euro;,3,F257-D28,157000.0
530592,100067196,2024-04-01,21926.0,3.0,1974.0,36023.0,549.0,135,44.6288,10.9141,...,Modena,Buono / Abitabile,G,Centralizzato,1 in box privato/box in garage,"centralizzato, a radiatori, alimentato a metano",200.001 - 300.000 &euro;,5,F257-D28,290000.0
530086,104685793,2024-04-01,99234.0,2.0,1909.0,17029.0,263.0,107,45.5316,10.2468,...,Brescia,Ottimo / Ristrutturato,E,Centralizzato,2 in parcheggio/garage comune,"centralizzato, a radiatori, alimentato a metano",300.001 - 500.000 &euro;,3,B157-C5,345000.0


In [ ]:
omi_df = omi_df[[
    'id_zona_omi',
    'tipologia',
    'condizione',
    'compr_min',
    'compr_max',
    'zona_omi_type',
]]

In [ ]:
omi_df

,id_zona_omi,tipologia,condizione,compr_min,compr_max,zona_omi_type
0,A048-B1,Abitazioni civili,normale,812.0,1165.0,B
1,A048-B1,Abitazioni civili,ottimo,1062.0,1454.0,B
2,A048-B1,Abitazioni civili,scadente,539.0,787.0,B
3,A048-B1,Abitazioni di tipo economico,normale,650.0,930.0,B
4,A048-B1,Abitazioni di tipo economico,ottimo,892.0,1192.0,B
...,...,...,...,...,...,...
245119,M422-E1,Abitazioni di tipo economico,ottimo,787.0,918.0,E
245120,M422-E1,Abitazioni di tipo economico,scadente,328.0,412.0,E
245121,M422-E1,Ville e Villini,normale,680.0,832.0,E
245122,M422-E1,Ville e Villini,ottimo,920.0,1050.0,E


## <span style="color:#ff5f27;"> 🪄 Creating Feature Groups </span>

In [15]:
fs = proj.get_feature_store()

In [16]:
# Get or create the 'properties' feature group
properties_fg = fs.get_or_create_feature_group(
    name='properties',
    version=4,                                                 # Breaking version
    description='Property Features and Corresponding Prices',
    online_enabled=True,                                       # Necessary for online inference
    primary_key=['id'],
    event_time='scraping_date',
)

In [17]:
# Insert data into feature group
properties_fg.insert(properties_df)

Uploading Dataframe: 100.00% |██████████| Rows 11232/11232 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: properties_4_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1158295/jobs/named/properties_4_offline_fg_materialization/executions


(Job('properties_4_offline_fg_materialization', 'SPARK'), None)

In [ ]:
omi_fg = fs.get_or_create_feature_group(
    name='omi',
    version=2,
    description='OMI information',
    online_enabled=True,
    primary_key=['id_zona_omi'],
)

In [ ]:
# Insert data into feature group
omi_fg.insert(omi_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1158295/fs/1148998/fg/1392972


Uploading Dataframe: 100.00% |██████████| Rows 240570/240570 | Elapsed Time: 00:14 | Remaining Time: 00:00


Launching job: omi_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1158295/jobs/named/omi_2_offline_fg_materialization/executions


(Job('omi_2_offline_fg_materialization', 'SPARK'), None)